In [80]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
import sklearn
import pandas as pd

In [97]:
df = pd.read_csv("filtered_labelled_features.csv")
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)
df.head()

,sequence,sid,pos_acts_toeachother,neg_acts_toeachother,pos_acts_together,neg_acts_together,pos_char1_acts,neg_char1_acts,pos_char2_acts,neg_char2_acts,...,neg,filename,unique_ID,29:posFramesFired,30:negFramesFired,31:otherFramesFired,32:posFramewrtCharFired,33:negFramewrtCharFired,34:otherFrameswrtCharFired,manualLabel
0,1,s0,0,0,0,0,0,0,0,1,...,1,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents0,2,2,1,1,2,1,p
1,1,s9,0,0,0,0,0,0,0,0,...,0,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents9,1,1,0,0,0,0,p
2,1,s10,0,0,0,0,0,0,1,0,...,0,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents10,2,1,0,0,0,0,p
3,1,s12,0,0,0,0,0,0,0,0,...,0,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents12,0,1,0,0,0,0,p
4,1,s13,0,0,0,0,0,0,0,0,...,2,aeschylus.libation.0.3.sent,aeschylus.libation.0.3.sents13,0,0,0,0,0,0,p


In [99]:
Y = df['manualLabel']
X = df.drop(['manualLabel', 'filename', 'unique_ID', 'sequence', 'sid'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .20, random_state = 1234)

In [100]:
logreg = linear_model.LogisticRegression()

# we create an instance of Neighbours Classifier and fit the data.
logreg.fit(X_train, y_train)

preds = logreg.predict(X_test)

correct = 0 
total = 0 

for pred, target in zip(preds, y_test):
    if pred == target:
        correct += 1
    total += 1
    
accuracy = correct/total 
print(accuracy)




0.8016528925619835


In [101]:
preds.shape

(121,)

In [102]:
results = pd.DataFrame()
print(preds.shape)
print(y_test.shape)
results['pred'] = preds
results['y_test'] = np.array(y_test)
results['correct'] = 0

(121,)
(121,)


In [103]:
results.head()

,pred,y_test,correct
0,p,p,0
1,p,p,0
2,p,p,0
3,p,p,0
4,p,p,0


In [104]:
results['correct'] = (results['pred'] == results['y_test']).astype(int)

In [105]:
results['y_test'].value_counts()

p    98
n    23
Name: y_test, dtype: int64

In [106]:
results['pred'].value_counts()

p    120
n      1
Name: pred, dtype: int64

In [107]:
results['correct'].sum()/results['correct'].count()

0.8016528925619835

In [108]:
preds = preds.astype('str')
y_test = y_test.astype('str')

In [109]:
from sklearn.metrics import average_precision_score

CF = sklearn.metrics.confusion_matrix(y_test.astype(str), preds)

labs = ['n', 'p']

FP = {}
for r, label in enumerate(labs):
    fp = 0
    for c in range(2):
        if r != c: 
            fp += CF[r][c]
        FP[label] = fp           
print(FP)


FN = {}
for c, label in enumerate(labs):
    fn = 0 
    for r in range(2):
        if c != r: 
            fn += CF[r][c]
        FN[label] = fn
print(FN)

TP = {}
for r, label in enumerate(labs):
    for c in range(2):
        if r == c:
            TP[label] = CF[r][c]

print(TP)

{'n': 23, 'p': 1}
{'n': 1, 'p': 23}
{'n': 0, 'p': 97}


In [110]:
# calculating precision and recall 

# precision = TP / TP + FP 

# recal = TP / TP + FN 

results = {'n': {}, 'p': {}}

for lab in labs:
    results[lab]['precision'] = TP[lab] / (TP[lab] + FP[lab])
    results[lab]['recall'] = TP[lab] / (TP[lab] + FN[lab])
    results[lab]['F1'] = 2 * ((results[lab]['precision'] * results[lab]['recall']) / (results[lab]['precision'] + results[lab]['recall']))
    

print(results)
    

{'n': {'precision': 0.0, 'recall': 0.0, 'F1': nan}, 'p': {'precision': 0.9897959183673469, 'recall': 0.8083333333333333, 'F1': 0.8899082568807339}}


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in double_scalars
  if sys.path[0] == '':


In [111]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy_score(y_test, preds)

0.8016528925619835

In [112]:
precision_score(y_test, preds, average=None)

array([0.        , 0.80833333])

In [113]:
recall_score(y_test, preds, average=None)

array([0.        , 0.98979592])

In [114]:
f1_score(y_test, preds, average=None)

array([0.        , 0.88990826])